In [1]:
import pandas as pd

In [2]:
zip_data = pd.read_csv('data/tests-by-zcta.csv')

In [3]:
zip_data.head()

,MODZCTA,Positive,Total,zcta_cum.perc_pos
0,NaN,4712,4716,99.92
1,10001.0,158,337,46.88
2,10002.0,360,718,50.14
3,10003.0,210,490,42.86
4,10004.0,20,45,44.44


In [4]:
zip_data.rename(columns = {'MODZCTA': 'zipcode', 'zcta_cum.perc_pos': '%_positive'}, inplace = True)

In [5]:
zip_data.head()

,zipcode,Positive,Total,%_positive
0,NaN,4712,4716,99.92
1,10001.0,158,337,46.88
2,10002.0,360,718,50.14
3,10003.0,210,490,42.86
4,10004.0,20,45,44.44


In [6]:
import folium
from folium import IFrame
import json
import folium.features as features

In [7]:
zip_geo = f'data/zipcode_polygons.json'

In [8]:
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)

In [9]:
zipcode_layer = folium.GeoJson(
zip_geo).add_to(m)

In [10]:
tip = features.GeoJsonTooltip(fields = ['postalCode']).add_to(zipcode_layer)

In [11]:
m

In [12]:
data = pd.read_csv('data/coronavirus_full_data.csv')

In [13]:
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
for i, v in data.iterrows():
    
    popup_text = data.iloc[i]['popup']
    popup = folium.Popup(popup_text, max_width = 300, min_width = 50)
    
    tooltip_text = '<b> ZipCode: </b>' + data.iloc[i]['Zip'].astype(str) + ' <br/> Click for more info.'
    tooltip = folium.Tooltip(tooltip_text)
    
    icon = features.DivIcon(icon_size = (50,50), icon_anchor = (0,0), html = '<b>' + data.iloc[i]['Positive'].astype(str) + '</b>')
    
    folium.Circle(tuple(data.iloc[i][['Latitude', 'Longitude']]), radius = float(data.iloc[i]['Positive']), tooltip = tooltip, popup= popup, fill = True, fill_color = '#FF0000', color = '#FF0000', icon = icon).add_to(m)
    #folium.map.Marker(tuple(data.iloc[i][['Latitude', 'Longitude']]), icon = icon).add_to(m)

In [14]:
m

In [113]:
data.head()

,Unnamed: 0,MODZCTA,Positive,Total,zcta_cum.perc_pos,Zip,City,State,Latitude,Longitude,geopoint,popup
0,0,10001.0,158,337,46.88,10001,New York,NY,40.750742,-73.99653,"40.750742,-73.99653",Zipcode: 10001\n Tested Positive: 158
1,1,10002.0,360,718,50.14,10002,New York,NY,40.717040,-73.98700,"40.71704,-73.987",Zipcode: 10002\n Tested Positive: 360
2,2,10003.0,210,490,42.86,10003,New York,NY,40.732509,-73.98935,"40.732509,-73.98935",Zipcode: 10003\n Tested Positive: 210
3,3,10005.0,29,96,30.21,10005,New York,NY,40.706019,-74.00858,"40.706019,-74.00858",Zipcode: 10005\n Tested Positive: 29
4,4,10006.0,7,30,23.33,10006,New York,NY,40.707904,-74.01342,"40.707904,-74.01342",Zipcode: 10006\n Tested Positive: 7


In [15]:
from xml.etree import ElementTree as ET

In [16]:
to_extract = ['HOUSEHOLD SIZE', 'HOUSING UNITS', 'RACE', 'SEX BY AGE', 'TOTAL POPULATION', 'HOUSEHOLD SIZE Percentage', 'RACE Percentage', 'SEX BY AGE Percentage' ]
data_dir = 'data/US Census/cleaned/'
df_dict = {}

for file_name in to_extract:
    df_dict[file_name] = pd.read_csv(data_dir + file_name + '.csv', index_col = 'MODZCTA')
    

In [105]:
def create_html_string(metrics: dict):
    html = ET.Element('html')
    w3_link = ET.Element('link', 
                         attrib = {'rel' : 'stylesheet', 
                                   'href' : r'https://www.w3schools.com/w3css/4/w3.css'})
    
    font_link = ET.Element('link', 
                           attrib= {'rel' : 'stylesheet', 'href': r'https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap'})
    style = ET.Element('style')
    style.text = r".w3-montserrat {font-family: 'Montserrat', sans-serif;}"
    
    html.append(w3_link)
    html.append(font_link)
    html.append(style)
    
    red_div = ET.Element('div', attrib = {'class': 'w3-container w3-red w3-montserrat'})
    html.append(red_div)
    
    zipcode_div = ET.Element('div', attrib = {'class' : 'w3-bar-item'})
    zipcode_div.text = 'Zipcode:'
    red_div.append(zipcode_div)
    zipvalue_div = ET.Element('div', attrib = {'class' : 'w3-bar-item'})
    bold_div = ET.Element('b')
    bold_div.text = str(metrics['Zip'])
    zipvalue_div.append(bold_div)
    red_div.append(zipvalue_div)
    
    body_keys = [k for k in metrics.keys() if k != 'Zip']
    for k in body_keys:
        new_div = ET.Element('div', attrib = {'class': 'w3-container w3-montserrat'})
        
        name_div = ET.Element('div', attrib = {'class' : 'w3-bar-item'})
        name_div.text = k
        
        value_div = ET.Element('div', attrib = {'class' : 'w3-bar-item'})
        bold_div = ET.Element('b')
        try:
            bold_div.text = f"{metrics[k]:,d}"
        except:
            bold_div.text = '{:,.2f}'.format(metrics[k])
                
        value_div.append(bold_div)
        
        new_div.append(name_div)
        new_div.append(value_div)
        html.append(new_div)
        
    return ET.tostring(html)


In [106]:
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
df = df_dict['TOTAL POPULATION']
for i, v in df.iterrows():
    
    metrics_dict = {}
    metrics_dict['Zip'] = df.loc[i]['Zip']
    
    metrics_dict['Tested Positive:'] = df.loc[i]['Positive']
    metrics_dict['Per 1,000:'] = round(df.loc[i]['Positive'] / df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION'] * 1000,2)
 
    metrics_dict['Population:'] = df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION']
   
    popup_text = create_html_string(metrics_dict).decode()
    #print(popup_text)
    popup = folium.Popup(popup_text, max_width = 150, min_width = 150)
    
    tooltip_text = '<b> ZipCode: </b>' + df.loc[i]['Zip'].astype(str) + ' <br/> Click for more info.'
    tooltip = folium.Tooltip(tooltip_text)
    
    icon = features.DivIcon(icon_size = (50,50), icon_anchor = (0,0), html = '<b>' + df.loc[i]['Positive'].astype(str) + '</b>')
    
    folium.Circle(tuple(df.loc[i][['Latitude', 'Longitude']]), radius = float(df.loc[i]['Positive']), tooltip = tooltip, popup= popup, fill = True, fill_color = '#FF0000', color = '#FF0000', icon = icon).add_to(m)
    #folium.map.Marker(tuple(data.iloc[i][['Latitude', 'Longitude']]), icon = icon).add_to(m)

In [107]:
m

In [122]:
def create_table_string(metrics: dict):
    html = ET.Element('html')
    w3_link = ET.Element('link', 
                         attrib = {'rel' : 'stylesheet', 
                                   'href' : r'https://www.w3schools.com/w3css/4/w3.css'})
    
    font_link = ET.Element('link', 
                           attrib= {'rel' : 'stylesheet', 'href': r'https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap'})
    style = ET.Element('style')
    style.text = r".w3-montserrat {font-family: 'Montserrat', sans-serif;}"
    
    html.append(w3_link)
    html.append(font_link)
    html.append(style)
    
    container = ET.Element('div', attrib = {'id':'summary','class': 'w3-container w3-montserrat'})
    html.append(container)
    
    table = ET.Element('table', attrib = {'class' : "w3-table w3-bordered w3-montserrat"})
    html.append(table)
    
    body_keys = [k for k in metrics.keys()]
    for k in body_keys:
        if k == "Zip Code:":
            new_row = ET.Element('tr', attrib = {'class': 'w3-red'})
        else:
            new_row = ET.Element('tr')
        name_cell = ET.Element('th')
        name_cell.text = k
        
        new_row.append(name_cell)
        
        value_cell = ET.Element('th')
        if k != 'Zip Code:':
            try:
                value_cell.text = f"{metrics[k]:,d}"
            except:
                value_cell.text = '{:,.2f}'.format(metrics[k])
        else:
            value_cell.text = str(metrics[k])
                
        new_row.append(value_cell)
        
        table.append(new_row)
        
    return ET.tostring(html)


In [123]:
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
df = df_dict['TOTAL POPULATION']
for i, v in df.iterrows():
    
    metrics_dict = {}
    metrics_dict['Zip Code:'] = df.loc[i]['Zip']
    
    metrics_dict['Tested Positive:'] = df.loc[i]['Positive']
    metrics_dict['Per 1,000:'] = round(df.loc[i]['Positive'] / df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION'] * 1000,2)
 
    metrics_dict['Population:'] = df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION']
   
    popup_text = create_table_string(metrics_dict).decode()
    #print(popup_text)
    popup = folium.Popup(popup_text, max_width = 150, min_width = 150)
    
    tooltip_text = '<b> ZipCode: </b>' + df.loc[i]['Zip'].astype(str) + ' <br/> Click for more info.'
    tooltip = folium.Tooltip(tooltip_text)
    
    icon = features.DivIcon(icon_size = (50,50), icon_anchor = (0,0), html = '<b>' + df.loc[i]['Positive'].astype(str) + '</b>')
    
    folium.Circle(tuple(df.loc[i][['Latitude', 'Longitude']]), radius = float(df.loc[i]['Positive']), tooltip = tooltip, popup= popup, fill = True, fill_color = '#FF0000', color = '#FF0000', icon = icon).add_to(m)
    #folium.map.Marker(tuple(data.iloc[i][['Latitude', 'Longitude']]), icon = icon).add_to(m)

In [124]:
m

In [17]:
def create_tabbed_string(metrics: dict, demos: dict):
    html = ET.Element('html')
    w3_link = ET.Element('link', 
                         attrib = {'rel' : 'stylesheet', 
                                   'href' : r'https://www.w3schools.com/w3css/4/w3.css'})
    
    font_link = ET.Element('link', 
                           attrib= {'rel' : 'stylesheet', 'href': r'https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap'})
    style = ET.Element('style')
    style.text = r".w3-montserrat {font-family: 'Montserrat', sans-serif;} header {border-radius: 12px 12px 0px 0px}"
    
    html.append(w3_link)
    html.append(font_link)
    html.append(style)
    
    header = ET.Element('header', attrib = {'class' : 'w3-container w3-red w3-montserrat'})
    html.append(header)
    h = ET.Element('h4')
    h.text = 'Zip Code:' + ' ' + str(metrics['Zip Code:'])
    
    header.append(h)
    
    '''create buttons'''
    
    button_div = ET.Element('div', attrib = {'class': 'w3-bar w3-black'})
    html.append(button_div)
    
    button = ET.Element('button', attrib = {'class': 'w3-bar-item w3-button', 'onclick':"openTab('Summary')"})
    button.text = 'Summary'
    button_div.append(button)
    
    button = ET.Element('button', attrib = {'class': 'w3-bar-item w3-button', 'onclick':"openTab('Demo')"})
    button.text = 'Demographics'
    button_div.append(button)
    
    '''container for the summary tab'''
    
    container = ET.Element('div', attrib = {'id':'Summary','class': 'w3-container table w3-montserrat w3-padding-0'})
    html.append(container)
    
    table = ET.Element('table', attrib = {'class' : "w3-table w3-bordered w3-montserrat"})
    container.append(table)
    
    body_keys = [k for k in metrics.keys() if k != 'Zip Code:']
    for k in body_keys:
        new_row = ET.Element('tr')
        name_cell = ET.Element('th')
        name_cell.text = k
        
        new_row.append(name_cell)
        
        value_cell = ET.Element('th')
        if k != 'Zip Code:':
            try:
                value_cell.text = f"{metrics[k]:,d}"
            except:
                value_cell.text = '{:,.2f}'.format(metrics[k])
        else:
            value_cell.text = str(metrics[k])
                
        new_row.append(value_cell)
        
        table.append(new_row)
        
    '''container for demographics tab'''
    
    demo_container = ET.Element('div', attrib = {'id':'Demo','class': 'w3-container table w3-montserrat w3-padding-0', 'style' : 'display:none'})
    html.append(demo_container)
    
    table = ET.Element('table', attrib = {'class' : "w3-table w3-bordered w3-montserrat w3-padding-0"})
    demo_container.append(table)
    
    tab_keys = [k for k in demos.keys()]
    for k in tab_keys:
        new_row = ET.Element('tr')
        if k == 'Race':
            name_cell = ET.Element('th')
            value_cell = ET.Element('th')
        else:
            name_cell = ET.Element('td')
            value_cell = ET.Element('td')
        #name_cell = ET.Element('td')
        name_cell.text = k
        
        new_row.append(name_cell)
        
        #value_cell = ET.Element('td')
        if not demos[k] == 'Percentage':
            value_cell.text = str(demos[k])
        else:
            value_cell.text = demos[k]
                
        new_row.append(value_cell)
        
        table.append(new_row)
    
    script = ET.Element('script')
    script.text = 'function openTab(tabName) { var i;  var x = document.getElementsByClassName("table");  for (i = 0; i < x.length; i++) {    x[i].style.display = "none";  } document.getElementById(tabName).style.display = "block";  }'
    html.append(script)
    
    return ET.tostring(html)

In [27]:
def metrics_organizer(df_, categories, ind, val, symbol):
    '''
    pulls dataframe values based on categories at a given index
    
    inputs: 
    df - Dataframe
    categories - dictionary used to rename column while preserving dataframe naming scheme
    ind - row index of df
    val - actual row of df at given ind
    symbol - symbol to append at each value
    
    returns:
    metrics - dictionary to feed into string-generator
    '''
    metrics = {}
    for key in categories.keys():
        metrics[key] = str(df_.loc[int(ind)][categories[key]]) + ' '+ symbol
    return metrics
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
zip_geo = f'data/zipcode_polygons.json'
style = {'fillColor': '#00000000', 'lineColor': '#00FFFFFF'}
zipcode_layer = folium.GeoJson(zip_geo, style_function = lambda x: style).add_to(m)
#tip = features.GeoJsonTooltip(fields = ['postalCode']).add_to(zipcode_layer)

df = df_dict['TOTAL POPULATION']
for i, v in df.iterrows():
    
    metrics_dict = {}
    metrics_dict['Zip Code:'] = df.loc[i]['Zip']
    
    metrics_dict['Tested Positive:'] = df.loc[i]['Positive']
    metrics_dict['Per 1,000:'] = round(df.loc[i]['Positive'] / df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION'] * 1000,2)
 
    metrics_dict['Population:'] = df_dict['TOTAL POPULATION'].loc[i]['Total_TOTAL POPULATION']
    
    demo_renaming = {'White':'White alone', 
                     'African American': 'Black or African American alone', 
                     'Native American': 'American Indian and Alaska Native alone', 
                     'Asian': 'Asian alone',
                     'Pacific Islander': 'Native Hawaiian and Other Pacific Islander alone',
                     'Other Races': 'Some Other Race alone',
                     'Two or More Races': 'Two or More Races'}
    
    demo_dict = {'Race' : 'Percentage'}
    demo_dict.update(metrics_organizer(df_dict['RACE Percentage'], demo_renaming, i, v, '%'))
    
    popup_text = create_tabbed_string(metrics_dict, demo_dict).decode().replace('&lt;', '<')
    iframe = IFrame(html=popup_text, width = '100%', height = 200)
    if i == 10001:
        sample = popup_text
    #print(popup_text)
    popup = folium.Popup(iframe, max_width = 300, min_width = 300)
    
    tooltip_text = '<b> ZipCode: </b>' + df.loc[i]['Zip'].astype(str) + ' <br/> Click for more info.'
    tooltip = folium.Tooltip(tooltip_text)
    
    icon = features.DivIcon(icon_size = (50,50), icon_anchor = (0,0), html = '<b>' + df.loc[i]['Zip'].astype(str) + '</b>')
    
    folium.Circle(tuple(df.loc[i][['Latitude', 'Longitude']]), radius = float(min(metrics_dict['Per 1,000:'] * 50, 1400)), tooltip = tooltip, popup= popup, fill = True, fill_color = '#FF0000', color = '#FF0000', icon = icon).add_to(m)
    #folium.map.Marker(tuple(data.iloc[i][['Latitude', 'Longitude']]), icon = icon).add_to(m)

In [28]:
m

In [20]:
m.save('master.html')

In [21]:
from bs4 import BeautifulSoup

In [22]:
soup = BeautifulSoup(open("master.html"))
links = soup.findAll('link')
for link in links:
    link['href'] = link['href'].replace(r"https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css", "https://sleepingtuna.github.io/NYCCovid-19Map.github.io/docs/css/leaflet.css")
with open("new_master.html", "w") as file:
    file.write(str(soup))

In [44]:
def create_bootstrap_string(metrics: dict, demos: dict):
    html = ET.Element('html')
    w3_link = ET.Element('link', 
                         attrib = {'rel' : 'stylesheet', 
                                   'href' : r'https://www.w3schools.com/w3css/4/w3.css'})
    
    font_link = ET.Element('link', 
                           attrib= {'rel' : 'stylesheet', 'href': r'https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap'})
    style = ET.Element('style')
    style.text = r".w3-montserrat {font-family: 'Montserrat', sans-serif;} header {border-radius: 12px 12px 0px 0px}"
    
    html.append(w3_link)
    html.append(font_link)
    html.append(style)
    
    header = ET.Element('header', attrib = {'class' : 'w3-container w3-red w3-montserrat'})
    html.append(header)
    h = ET.Element('h4')
    h.text = 'Zip Code:' + ' ' + str(metrics['Zip Code:'])
    
    header.append(h)
    
    '''create buttons'''
    
    button_div = ET.Element('div', attrib = {'class': 'w3-bar w3-black'})
    html.append(button_div)
    
    button = ET.Element('button', attrib = {'class': 'w3-bar-item w3-button', 'onclick':"openTab('Summary')"})
    button.text = 'Summary'
    button_div.append(button)
    
    button = ET.Element('button', attrib = {'class': 'w3-bar-item w3-button', 'onclick':"openTab('Demo')"})
    button.text = 'Demographics'
    button_div.append(button)
    
    '''container for the summary tab'''
    
    container = ET.Element('div', attrib = {'id':'Summary','class': 'w3-container table w3-montserrat w3-padding-0'})
    html.append(container)
    
    table = ET.Element('table', attrib = {'class' : "w3-table w3-bordered w3-montserrat"})
    container.append(table)
    
    body_keys = [k for k in metrics.keys() if k != 'Zip Code:']
    for k in body_keys:
        new_row = ET.Element('tr')
        name_cell = ET.Element('th')
        name_cell.text = k
        
        new_row.append(name_cell)
        
        value_cell = ET.Element('th')
        if k != 'Zip Code:':
            try:
                value_cell.text = f"{metrics[k]:,d}"
            except:
                value_cell.text = '{:,.2f}'.format(metrics[k])
        else:
            value_cell.text = str(metrics[k])
                
        new_row.append(value_cell)
        
        table.append(new_row)
        
    '''container for demographics tab'''
    
    demo_container = ET.Element('div', attrib = {'id':'Demo','class': 'w3-container table w3-montserrat w3-padding-0', 'style' : 'display:none'})
    html.append(demo_container)
    
    table = ET.Element('table', attrib = {'class' : "w3-table w3-bordered w3-montserrat w3-padding-0"})
    demo_container.append(table)
    
    tab_keys = [k for k in demos.keys()]
    for k in tab_keys:
        new_row = ET.Element('tr')
        if k == 'Race':
            name_cell = ET.Element('th')
            value_cell = ET.Element('th')
        else:
            name_cell = ET.Element('td')
            value_cell = ET.Element('td')
        #name_cell = ET.Element('td')
        name_cell.text = k
        
        new_row.append(name_cell)
        
        #value_cell = ET.Element('td')
        if not demos[k] == 'Percentage':
            value_cell.text = str(demos[k])
        else:
            value_cell.text = demos[k]
                
        new_row.append(value_cell)
        
        table.append(new_row)
    
    script = ET.Element('script')
    script.text = 'function openTab(tabName) { var i;  var x = document.getElementsByClassName("table");  for (i = 0; i < x.length; i++) {    x[i].style.display = "none";  } document.getElementById(tabName).style.display = "block";  }'
    html.append(script)
    
    return ET.tostring(html)

[<link href="css/leaflet.css" rel="stylesheet"/>,
 <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css" rel="stylesheet"/>,
 <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" rel="stylesheet"/>,
 <link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" rel="stylesheet"/>,
 <link href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css" rel="stylesheet"/>,
 <link href="https://rawcdn.githack.com/python-visualization/folium/master/folium/templates/leaflet.awesome.rotate.css" rel="stylesheet"/>]